INTRO:
This code happens after the data transformation i.e. after missings are imputed etc.
(No worries, we'll check with that undrstanding)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import os
import re
from collections import Counter
# import statsmodels.api as sm
from xgboost import XGBClassifier
#import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc, precision_score, recall_score, f1_score, confusion_matrix
from datetime import datetime 
from sklearn.model_selection import RandomizedSearchCV
import time
# from shapely import wkt
import warnings
warnings.filterwarnings('ignore')

# Not importing lists bc run lists script below
#import lists# Long-ass lists of vars etc. (keep in same folder)
import utils# Define util functions

%matplotlib inline
%run lists.py

pd.options.display.max_rows = 1000

Loading lists of vars and printing out some list sizes...
75
45


In [2]:
FPATH = "/Users/david.duong/dev/lending-club/"
FNAME= "post_fe_dat.csv"


In [10]:
dat = pd.read_csv(FPATH + FNAME)
print(dat.shape)

(1023205, 123)


In [11]:
dat.head()

,id,Default_flag,issue_ym,highRiskZip,major_derog_NA,bc_dlq_NA,revol_delinq_NA,grade_a,grade_b,grade_c,...,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,total_cu_tl,inq_last_12m
0,68407277,0,201512,0,0,0,0,0.0,0.0,1.0,...,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,3.0,1.0,4.0
1,68355089,0,201512,0,1,1,0,0.0,0.0,1.0,...,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,0.0,0.0,6.0
2,68426831,0,201512,0,1,1,1,0.0,0.0,1.0,...,338.0,3976.0,99.0,0.0,0.0,4522.0,76.0,0.0,0.0,0.0
3,68476668,0,201512,0,1,1,1,0.0,1.0,0.0,...,18.0,29433.0,63.0,2.0,3.0,13048.0,74.0,1.0,0.0,1.0
4,67275481,0,201512,0,0,1,1,0.0,1.0,0.0,...,13.0,27111.0,75.0,0.0,0.0,640.0,55.0,1.0,0.0,2.0


In [19]:
dat[numeric_cols_4_model + ['issue_ym']].groupby('issue_ym').quantile([.05, .25, .5, .75, .95])

acc_now_delinq  acc_open_past_24mths  all_util   annual_inc  \
issue_ym                                                                     
200706   0.05             0.0                   0.0      0.00    6075.0000   
         0.25             0.0                   0.0      0.00   20000.0000   
         0.50             0.0                   0.0      0.00   33500.0000   
         0.75             0.0                   0.0      0.00   61656.0000   
         0.95             0.0                   0.0      0.00  169500.0000   
200707   0.05             0.0                   0.0      0.00    8160.0000   
         0.25             0.0                   0.0      0.00   22500.0000   
         0.50             0.0                   0.0      0.00   50000.0000   
         0.75             0.0                   0.0      0.00   85500.0000   
         0.95             0.0                   0.0      0.00  180000.0000   
200708   0.05             0.0                   0.0      0.00    7182.0000   
         0.25             0.0                   0.0      0.00   25750.0000   
         0.50             0.0                   0.0      0.00   58500.0000   
         0.75             0.0                   0.0      0.00   96000.0000   
         0.95             0.0                   0.0      0.00  217200.0000   
200709   0.05             0.0                   0.0      0.00   14539.6000   
         0.25             0.0                   0.0      0.00   30000.0000   
         0.50             0.0                   0.0      0.00   50000.0000   
         0.75             0.0                   0.0      0.00   75000.0000   
         0.95             0.0                   0.0      0.00  132000.0000   
200710   0.05             0.0                   0.0      0.00   14520.0000   
         0.25             0.0                   0.0      0.00   29000.0000   
         0.50             0.0                   0.0      0.00   43500.0000   
         0.75             0.0                   0.0      0.00   65000.0000   
         0.95             0.0                   0.0      0.00  120000.0000   
200711   0.05             0.0                   0.0      0.00   14310.0000   
         0.25             0.0                   0.0      0.00   27000.0000   
         0.50             0.0                   0.0      0.00   48356.5000   
         0.75             0.0                   0.0      0.00   75000.0000   
         0.95             0.0                   0.0      0.00  159000.0000   
200712   0.05             0.0                   0.0      0.00   19473.0000   
         0.25             0.0                   0.0      0.00   36000.0000   
         0.50             0.0                   0.0      0.00   50876.0000   
         0.75             0.0                   0.0      0.00   80000.0000   
         0.95             0.0                   0.0      0.00  174000.0000   
200801   0.05             0.0                   0.0      0.00   18000.0000   
         0.25             0.0                   0.0      0.00   35000.0000   
         0.50             0.0                   0.0      0.00   50000.0000   
         0.75             0.0                   0.0      0.00   73992.0000   
         0.95             0.0                   0.0      0.00  138600.0000   
200802   0.05             0.0                   0.0      0.00   21700.0000   
         0.25             0.0                   0.0      0.00   36000.0000   
         0.50             0.0                   0.0      0.00   50716.5000   
         0.75             0.0                   0.0      0.00   75000.0000   
         0.95             0.0                   0.0      0.00  146625.0000   
200803   0.05             0.0                   0.0      0.00   21575.0000   
         0.25             0.0                   0.0      0.00   37769.0000   
         0.50             0.0                   0.0      0.00   52000.0000   
         0.75             0.0                   0.0      0.00   78000.0000   
         0.95             0.0        

## A. Review Target

In [14]:
for col in numeric_cols_4_model[:3]:
    print(col)
    print(dat.groupby('issue_ym'))
    #print(np.percentile(dat[col].values, [0, 25, 50, 75, 100]))
    

loan_amnt


AttributeError: Cannot access attribute 'values' of 'SeriesGroupBy' objects, try using the 'apply' method

## B. Review continuous features

## C. Review cat features